In [46]:
import os
import glob

import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import cv2

In [83]:
classes = ["glioma_tumor", "meningioma_tumor", "pituitary_tumor"]

In [84]:
# öncelikle training verilerini klasörden okuyacağız.
image_dir = "dataset/training/images"
label_dir = "dataset/training/labels"

augmented_image_dir = "augmented_dataset/training/images"
augmented_label_dir = "augmented_dataset/training/labels"

image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

In [85]:

def get_transform(width, height):
    transform = A.Compose(
        [
            A.OneOf([
                A.RandomCrop(width=int(width * 0.8), height=int(height * 0.8), p=0.9),
                A.ShiftScaleRotate(p=0.9, border_mode=cv2.BORDER_CONSTANT, rotate_limit=15),
            ], p=0.9),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.9),
                A.RandomGamma(p=0.9),
            ]),
        ], bbox_params=A.BboxParams(format="yolo", min_visibility=0.5)
    )

    return transform

In [86]:
# Elimizde her sınıf için yaklaşık 350 örnek var ve bu yetersiz bu sebep ile veri artırma yapacağız. Her örnek için yeni 2 örnek oluşturacağız.

for image_path in image_paths:

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = get_transform(image.shape[1], image.shape[0])

    label_path = os.path.join(label_dir, image_path.split("/")[-1].split(".")[0] + ".txt")
    
    with open(label_path, "r") as file:
        data = file.read().split("\n")
    
    bboxes = []
    for line in data:

        if line:
            label_idx, normalized_x_center, normalized_y_center, normalized_width, normalized_height = line.split(" ")
            bboxes.append([float(normalized_x_center), float(normalized_y_center), float(normalized_width), float(normalized_height), classes[int(label_idx)]])

    for i in range(2):

        # Bounding box koordinatlarını alıp veri artırma işlemi yapacağız.
        augmented = transform(image=image, bboxes=bboxes)
        augmented_image = augmented["image"]
        augmented_bboxes = augmented["bboxes"]
        

        # Veriyi kaydetme
        augmented_image_name = image_path.split("/")[-1].split(".")[0] + f"_augmented_{i+1}.jpg"
        augmented_label_name = image_path.split("/")[-1].split(".")[0] + f"_augmented_{i+1}.txt"

        # # draw bounding boxes
        # for bbox in augmented_bboxes:
        #     x_center, y_center, width, height, label = bbox
        #     x_min = int((x_center - width / 2) * augmented_image.shape[1])
        #     y_min = int((y_center - height / 2) * augmented_image.shape[0])
        #     x_max = int((x_center + width / 2) * augmented_image.shape[1])
        #     y_max = int((y_center + height / 2) * augmented_image.shape[0])

        #     cv2.rectangle(augmented_image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        #     cv2.putText(augmented_image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        cv2.imwrite(os.path.join(augmented_image_dir, augmented_image_name), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

        for bbox in augmented_bboxes:

            x_center, y_center, width, height, label = bbox
            
            with open(os.path.join(augmented_label_dir, augmented_label_name), "a") as file:
                file.write(f"{classes.index(label)} {x_center} {y_center} {width} {height}\n")

    # # draw bounding boxes
    # for bbox in bboxes:
    #     x_center, y_center, width, height, label = bbox
    #     x_min = int((x_center - width / 2) * image.shape[1])
    #     y_min = int((y_center - height / 2) * image.shape[0])
    #     x_max = int((x_center + width / 2) * image.shape[1])
    #     y_max = int((y_center + height / 2) * image.shape[0])

    #     cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
    #     cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # save original image and label
    cv2.imwrite(os.path.join(augmented_image_dir, image_path.split("/")[-1]), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    with open(os.path.join(augmented_label_dir, label_path.split("/")[-1]), "w") as file:
        file.write(f"{classes.index(label)} {normalized_x_center} {normalized_y_center} {normalized_width} {normalized_height}")
